<a href="https://colab.research.google.com/github/deepikaparasa6/SmartWasteManagement/blob/develop/SmartWasteManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python tensorflow fastapi uvicorn streamlit kaggle matplotlib


In [ ]:
from google.colab import files
files.upload()  # Upload your kaggle.json API key
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d asdasdasasdas/garbage-classification
!unzip garbage-classification.zip -d garbage_data


In [ ]:
import os
base_dir = "garbage_data/Garbage classification/Garbage classification"
print("Classes available:", os.listdir(base_dir))


In [ ]:
import tensorflow as tf

batch_size = 32
img_size = (224, 224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Class names:", class_names)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

image_path = f'{base_dir}/plastic/plastic1.jpg'
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title("Plastic Waste Sample")
plt.axis('off')
plt.show()


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(
    buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
classes = os.listdir(base_dir)
num_classes = len(classes)
print(f"We have {num_classes} classes")
total_images = 0
for c in classes:
    images = os.listdir(os.path.join(base_dir, c))
    total_images += len(images)
    print(f"for category {c}, we have {len(images)} images")

print(f"Total Images are: {total_images}")

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(input_shape=img_size + (3,),include_top=False,
                         weights='imagenet')

base_model.trainable = False  # Freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax')  # 6 classes
])


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=25)


In [ ]:
model.save('waste_classifier_model.h5')
